In [2]:
import gym
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)


Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])


def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)
        if is_done:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs


def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for reward, steps in batch:
        if reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, steps))
        train_act.extend(map(lambda step: step.action, steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [4]:
env = gym.make("CartPole-v0")
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-cartpole")

for iter_no, batch in enumerate(iterate_batches(
        env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = \
        filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f" % (
        iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    if reward_m > 199:
        print("Solved!")
        break
writer.close()

C:\Users\AS\AppData\Local\Temp/ipykernel_6800/4041599821.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  obs_v = torch.FloatTensor([obs])


0: loss=0.692, reward_mean=22.2, rw_bound=23.5
1: loss=0.669, reward_mean=22.7, rw_bound=27.5
2: loss=0.657, reward_mean=41.2, rw_bound=51.0
3: loss=0.654, reward_mean=37.4, rw_bound=48.5
4: loss=0.642, reward_mean=47.8, rw_bound=54.5
5: loss=0.644, reward_mean=36.2, rw_bound=41.5
6: loss=0.632, reward_mean=49.3, rw_bound=49.0
7: loss=0.614, reward_mean=41.5, rw_bound=45.5
8: loss=0.632, reward_mean=47.5, rw_bound=57.0
9: loss=0.624, reward_mean=46.2, rw_bound=56.5
10: loss=0.602, reward_mean=56.0, rw_bound=60.5
11: loss=0.602, reward_mean=64.1, rw_bound=66.0
12: loss=0.610, reward_mean=56.2, rw_bound=56.5
13: loss=0.591, reward_mean=57.2, rw_bound=71.5
14: loss=0.588, reward_mean=68.2, rw_bound=80.0
15: loss=0.591, reward_mean=64.8, rw_bound=74.5
16: loss=0.581, reward_mean=72.9, rw_bound=88.0
17: loss=0.581, reward_mean=54.9, rw_bound=65.5
18: loss=0.574, reward_mean=69.1, rw_bound=68.0
19: loss=0.583, reward_mean=78.0, rw_bound=71.5
20: loss=0.598, reward_mean=67.4, rw_bound=70.5
21